In [494]:
#CRAWLING DATA FROM YAHOO MOVIES
import requests
import re
from bs4 import BeautifulSoup
import json

Y_MOVIE_URL = "https://movies.yahoo.com.tw/movieinfo_main/"

class MovieCrawler(object):

    def __init__(self):
        self.session = requests.Session()
        
    def get_movies(self, page_url):
        movies = []
          # with open(page_url,'r') as f:
          #   contents = f.read()
        for i in range(2,3):
            response = self.session.get(page_url+str(i))#+"?page="+str(i))
            response.encoding = 'utf-8'
            soup = BeautifulSoup(response.text, 'html.parser')
            movie_list = soup.find_all('div', id='content_l')
            # print(movie_list)
            for movie in movie_list:
                label_class = []
                # print(movie)
                try:
                    ch_name = movie.find('div', class_='movie_intro_info_r').h1#.text.strip()
                    try:
                        ch_name.span.decompose()
                    except:
                        pass
                    ch_name = ch_name.text.strip()
                    en_name = movie.find('div', class_='movie_intro_info_r').h3.text.strip()
                    release_date = movie.find('div', class_='movie_intro_info_r').find('span',class_=None).text[5:].strip()#.span.text.split('上映日期：')[-1].strip()
                    labels = movie.find_all('div',class_ = 'level_name')
                    for label in labels:
                        try:
                            lb = label.find('a',class_='gabtn').text.strip()
                        except:
                            break
                        label_class.append(lb)
                    intro = movie.find('div', class_='gray_infobox_inner').span.text.strip().replace('\n','')
                    doc_id = i
                    movie_info = {
                                'doc_id' : doc_id,
                                'ch_name': ch_name,
                                'en_name': en_name,
                                'label[class]': label_class,
                                'release_date': release_date,
                                'intro': intro}
                    movies.append(movie_info)
                except:
                    print(i)
                    continue
        return movies
    
    def savejson(self,data):
        with open("./test.json", "w",encoding = 'utf-8') as final:
            json.dump(data, final,ensure_ascii=False)
        final.close()

    def addjson(self,data):
        with open("./test.json", "r",encoding='utf-8') as fp:
            listObj = json.load(fp)
            for m in data:
                listObj.append(m)
        print(len(listObj))
        fp.close()
        with open("./test.json", "w",encoding = 'utf-8') as final:
            json.dump(listObj, final,ensure_ascii=False)
        final.close()
        
    def readjson(self,data):
        with open(f"./{data}.json", "r",encoding = 'utf-8') as final:
            listObj = json.load(final)
        final.close()
        return listObj
            

In [495]:
crawler = MovieCrawler()
movie = crawler.get_movies(Y_MOVIE_URL)
# crawler.savejson(movie)
listObj = crawler.readjson("hw2")
# THE RESULTS : AS THE FOLLOWING SECTION
# {'ch_name', 'en_name', 'movie_url', 'release_date', 'intro'}
# print(len(movie))
# print(*movie, sep="\n")

In [534]:

class search_engine():
        
    def query(text):
        count_all = 0
        count = 0
        movie_prl = []
        movie_list = []
        #find all movie related to text
        regex = re.compile(rf'(.*){text}')
        for movie,data in movie_data.items():
            if regex.match(dict(data)['ch_name']) or regex.match(dict(data)['en_name']) or regex.match(dict(data)['intro']):
                count_all += 1
        for movie,index in inverted_index.items():
            if regex.match(str(movie)):
                for i in index:
                    movie_list.append(i[0])
        #print all movies related with text
        movies_list=list(dict.fromkeys(movie_list))
        data = []
        for ID in movies_list:
            if regex.match(movie_data[ID]['ch_name']) or regex.match(movie_data[ID]['en_name']) or regex.match(movie_data[ID]['intro']):
                count+=1
            data.append([movie_data[ID]['doc_id'],movie_data[ID]['pagerank'],movie_data[ID]['ch_name'], movie_data[ID]['en_name'],movie_data[ID]['intro']])
        data.sort(key = lambda x:x[1],reverse = True)
        print(f"There are {len(movies_list)} matches: ")
        for movie in data:
            print("DOC_ID:",movie[0])
            print("CHINESE TITLE:",movie[2])
            print("ENGLISH TITLE:",movie[3])
            print("PAGERANK:",movie[1])
            print("INTRO:",movie[4])
            print("\n")
        print("Precision = "+ str(count/len(movies_list)*100)+"%")
        print("Recall = "+str(count/count_all*100)+"%")
        print(count)
        print(count_all)
        print(len(movies_list))
        

# INVERTED INDEX

In [ ]:
from collections import Counter
import networkx as nx
import jieba
import re
from zhon.hanzi import punctuation
stopwords = [line.strip().replace('\n','') for line in open('../hw1/stopwords.txt','r').readlines()]
punctuations = [line.strip().replace('\n','') for line in open('../hw1/punctuation.txt','r').readlines()]
jieba.load_userdict('../hw1/userDict.txt')
punctuation = list(punctuation)

intro_all = []
for content in listObj:
    intro = content['intro'].replace('\n','').replace('\t','').replace('\u3000','')
    intro = re.sub('[^\u4e00-\u9fa5]','',intro)
    intro = jieba.lcut(intro)
    Intro = []
    for word in intro:
        if word not in stopwords:
            Intro.append(word)
    Intro.insert(0,content['ch_name'])
    Intro.insert(0,content['doc_id'])
    intro_all.append(Intro)

words = {}
for intro in intro_all:
    for word in intro:
        if word in words:
            words[word].append(intro[0])
        else:
            words[word] = [intro[0]]
            
from collections import Counter
inverted_index = {}
for i,j in words.items():
#     print(i)
    wordcount = (Counter(j))
    list(wordcount.items()).sort(key = lambda x : x[1],reverse = True)
    wordcounts = list(wordcount.items())
    inverted_index[i] = wordcounts

# PAGERANK

In [498]:
import networkx as nx
G = nx.DiGraph()
Edges = []
for word,doc_id in inverted_index.items():
    edges = []
    for ID in doc_id[2:]:
        if ID[0] != doc_id[0][0]:
            edges.append((doc_id[0][0],ID[0]))
        Edges.append(edges)

for edges in Edges:
    for edge in edges:
        G.add_edge(edge[0],edge[1])

pagerank_list = nx.pagerank(G,alpha = 1)
# print(pagerank_list)

PageRankList = list(pagerank_list.items())
PageRankList.sort(key = lambda x:x[0])

# CREATING DOC LINKS

In [499]:
DOC_LINK = {}
doc_link = []
i = 0
for data in inverted_index.values():
    id_num = intro_all[i][0] 
    doc_id = data[0][0]
    for ID in data:
        if ID[0] not in doc_link and ID[0]!= doc_id:
            doc_link.append(ID[0])
    if doc_id != id_num:
        i+=1
        doc_link.sort()
        DOC_LINK[id_num] = doc_link
        doc_link = []
    

# SAVING DATA into json file hw2.json

In [503]:
DATA = []
for listobj in listObj:
    try:
        d_link = DOC_LINK[dict(listobj)['doc_id']]
    except:
        d_link = "None"
    try:
        prl = dict(PageRankList)[dict(listobj)['doc_id']]
    except:
        prl = 0
    info = {
        'doc_id' : dict(listobj)['doc_id'],
        'ch_name' : dict(listobj)['ch_name'],
        'en_name' : dict(listobj)['en_name'],
        'pagerank' : prl,
        'label[class]' : dict(listobj)['label[class]'],
        'intro' : dict(listobj)['intro'],
        'release_date' : dict(listobj)['release_data'],
        'links(doc_id)' : d_link,
    }
#     print(dict(listobj)['doc_id'])
    DATA.append(info)
with open("./test.json", "w",encoding = 'utf-8') as final:
    json.dump(DATA, final,ensure_ascii=False)
final.close()

# Turn data into dictionary with id as key

In [504]:
movie_data = {}
for movie in DATA:
    movie_data[dict(movie)['doc_id']] = movie

# QUERY

In [536]:
search_engine.query("葉問")

There are 33 matches: 
DOC_ID: 6169
CHINESE TITLE: 華語大排檔
ENGLISH TITLE: 
PAGERANK: 0.00032226289840486995
INTRO: ★集合經典賀歲《家有囍事》《哪一天我們會飛》《惡人谷》《北京愛情故事》《我來自紐約》《愛在深秋》《王家欣》《開飯啦》《我們停戰吧！》9部大片。★周星馳、張國榮、張曼玉、吳君如、劉嘉玲、梁家輝、歐陽娜娜、楊千嬅、吳鎮宇、溫碧霞、鄭中基、杜汶澤、蘇麗珊、譚耀文、狄龍等實力派演員大集合。★真情動人、歡樂爆笑、惡搞KUSO、影展強片、懸疑驚悚還有難忘經典大作，各種類型齊備，千萬不能錯過話題作品 精彩可期《家有囍事》修復加長版導演：《開心鬼》《富貴逼人》高志森演員：《功夫》《少林足球》周星馳《倩女幽魂》《金枝玉葉》張國榮《甜蜜蜜》《花樣年華》張曼玉《歲月神偷》《金雞》吳君如《開心鬼》《八星報喜》黃百鳴《龍的傳人》《花田囍事》毛舜筠類型：爆笑喜劇上映日期：3月11日關於《家有囍事》1992年推出的賀歲電影《家有囍事》集合傳奇卡斯周星馳、張國榮、張曼玉、吳君如、毛舜筠香港喜劇高手，再加上鬼才編劇黃百鳴及喜戲名導高志森，讓電影未上映就轟動，甚至傳出「半夜搶底片」的社會案件，一上映果然開出紅盤，在港台都創下票房紀錄。《家有囍事》中各式無厘頭的搞笑對白及劇情，才是此片成為賀歲經典的主因；像是周星馳生病病名「無定向喪心病狂歇些性全身機能失調症」，張國榮嬌喊「生孩子太痛了」，張曼玉的「巴黎鐵塔反轉再反轉」接吻，吳君如大唱＜相逢何必成相識＞，都成為影迷難忘的喬段，對影迷生活和影壇都有著絕對的影響力，可說是空前絕後的賀歲大作。今年推出的《家有囍事》的修復加長版，加入了當年被修剪喬段；常家三兄弟頭帶絲襪與兇徒展開生厘頭的火拼槍戰及「有夠沒必要」的爆破場面，讓電影噴飯程度再加成，在24年後，帶給粉絲不同的新意。【故事大綱】常家三兄弟與父母同住在祖產大屋中。老大常滿（黃百鳴 飾）有事業有成，太太程氏（吳君如 飾）因多年來穩定的婚姻生活而變得不修邊幅，常滿在外找了美麗情婦Sheila。因一次約會後酒醉，Sheila送常滿回家時被程氏發現，一怒之下出走。老二常騷（張國榮 飾）是女性化的花美男，總是與男性化的表姑媽梁無雙（毛舜筠 飾）鬥氣，打牌、工作都能爭吵不休，鬧出不少笑話。老三常